## Goals

* Record data
* Prepare input layer
* Create a layer with the necessary convolutions
* create the output layer
* create the sequential model of nvidia using the prepared model
* Compile the model using the right optimizer and loss calculation function
* Split the data into the train set and validation set
* Use generator to pre-process the images
* train the model with batch size, number of epochs and model fit generator

Input for learning is the images and the prediction is the steering angle

In [ ]:
# Read data
import csv
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
from keras.models import Sequential
from keras.layers import Input,Flatten,Dense,Lambda,Cropping2D,GlobalAveragePooling2D,Convolution2D
from keras.preprocessing.image import ImageDataGenerator
from sklearn.utils import shuffle
import sklearn


model = Sequential()
model.add(Lambda(lambda x: x/255.0 - 0.5,input_shape=(160,320,3)))
model.add(Cropping2D(cropping=((50,20), (0,0))))
model.add(Convolution2D(24,(5,5),subsample=(2,2),activation='relu'))
model.add(Convolution2D(36,(5,5),subsample=(2,2),activation='relu'))
model.add(Convolution2D(48,(5,5),subsample=(2,2),activation='relu'))
model.add(Convolution2D(64,(3,3),activation='relu'))
model.add(Convolution2D(64,(3,3),activation='relu'))
model.add(Flatten())
model.add(Dense(100))
model.add(Dense(50))
model.add(Dense(10))
model.add(Dense(1))

model.summary()

model.compile(loss='mse',optimizer='adam',metrics=['accuracy'])

# Train the model
batch_size = 36
epochs = 5

samples =[]
with open('data/driving_log.csv') as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        samples.append(row)

from sklearn.model_selection import train_test_split
train_samples, validation_samples = train_test_split(samples, test_size=0.2)

def generator(samples, batch_size=32):
    num_samples = len(samples)
    while 1: # Loop forever so the generator never terminates
        shuffle(samples)
        for offset in range(0, num_samples, batch_size):
            batch_samples = samples[offset:(offset+batch_size//3)]
            images = []
            measurements = []
            
            for batch_sample in batch_samples:
                steering_center = float(row[3])
                correction = 0.25
                steering_left = steering_center + correction
                steering_right = steering_center - correction
        
                for i in range(3):
                    source_path = row[i]
                    filename = source_path.split('/')[-1]
                    current_path = 'data/IMG/'
                    if i==0:
                        img = np.asarray(mpimg.imread(current_path + filename))
                        images.append(img)
                        measurements.append(steering_center)
                        images.append(np.fliplr(img))
                        measurements.append(steering_center*(-1))
                    elif i==1:
                        img = np.asarray(mpimg.imread(current_path + filename))
                        images.append(img)
                        measurements.append(steering_left)
                        images.append(np.fliplr(img))
                        measurements.append(steering_left*(-1))
                    elif i==2:
                        img = np.asarray(mpimg.imread(current_path + filename))
                        images.append(img)
                        measurements.append(steering_right)
                        images.append(np.fliplr(img))
                        measurements.append(steering_left*(-1))
                    
            X_train = np.array(images)
            y_train = np.array(measurements)
                
            yield shuffle(X_train, y_train)

train_generator = generator(train_samples, batch_size=batch_size)
validation_generator = generator(validation_samples, batch_size=batch_size)

# Note: we aren't using callbacks here since we only are using 5 epochs to conserve GPU time
object_X1 = model.fit_generator(train_generator, steps_per_epoch=np.ceil(len(train_samples)/batch_size),
                                epochs=epochs,verbose=1,validation_data=validation_generator,
                                validation_steps=np.ceil(len(validation_samples)/batch_size))

print(object_X1.history.keys())
### plot the training and validation loss for each epoch
plt.plot(object_X1.history['loss'])
plt.plot(object_X1.history['val_loss'])
plt.title('model mean squared error loss')
plt.ylabel('mean squared error loss')
plt.xlabel('epoch')
plt.legend(['training set', 'validation set'], loc='upper right')
plt.show()

model.save('model.h5')

In [2]:
# Read data
import csv
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
from keras.models import Sequential
from keras.layers import Input,Flatten,Dense,Lambda,Cropping2D,GlobalAveragePooling2D,Convolution2D
from keras.preprocessing.image import ImageDataGenerator
from sklearn.utils import shuffle
import sklearn

samples =[]
with open('data_hemanth/driving_log.csv') as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        samples.append(row)
print(len(samples))

C:\Users\Kishan\Miniconda3\envs\carnd-term1\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


8741


In [3]:
print(samples[0])
from sklearn.model_selection import train_test_split
train_samples, validation_samples = train_test_split(samples, test_size=0.2)
offset = 0

['/home/workspace/CarND-Behavioral-Cloning-P3/Data_hemanth/IMG/center_2020_06_19_19_13_05_286.jpg', '/home/workspace/CarND-Behavioral-Cloning-P3/Data_hemanth/IMG/left_2020_06_19_19_13_05_286.jpg', '/home/workspace/CarND-Behavioral-Cloning-P3/Data_hemanth/IMG/right_2020_06_19_19_13_05_286.jpg', '0', '0', '0', '1.517092E-06']


In [28]:
def generator(samples, batch_size=18):
    num_samples = len(samples)
    while 1: # Loop forever so the generator never terminates
        shuffle(samples)
        for offset in range(0, num_samples, batch_size):
            batch_samples = samples[offset:(offset+batch_size)]
            print(offset)
            images = []
            measurements = []
            
            for batch_sample in batch_samples:
                steering_center = float(batch_sample[3])
                correction = 0.10
                steering_left = steering_center + correction
                steering_right = steering_center - correction
                
                for i in range(3):
                    source_path = batch_sample[0]
                    filename = source_path.split('/')[-1]
                    current_path = 'Data_hemanth/IMG/'
                    if i==0:
                        img = np.asarray(mpimg.imread(current_path + filename))
                        images.append(img)
                        measurements.append(steering_center)
                        images.append(np.fliplr(img))
                        measurements.append(steering_center*(-1))
                
                    elif i==1:
                        img = np.asarray(mpimg.imread(current_path + filename))
                        images.append(img)
                        measurements.append(steering_left)
                        images.append(np.fliplr(img))
                        measurements.append(steering_left*(-1))
                    elif i==2:
                        img = np.asarray(mpimg.imread(current_path + filename))
                        images.append(img)
                        measurements.append(steering_right)
                        images.append(np.fliplr(img))
                        measurements.append(steering_right*(-1))
                
                    
                X_train = np.array(images)
                y_train = np.array(measurements)
                
                yield shuffle(X_train, y_train)
batch_size=18

def train_generator():
    while(True):
        yield generator(train_samples, batch_size=batch_size)

def validation_generator():
    while(True):
        yield generator(validation_samples, batch_size=batch_size)

In [30]:
X_train, y_train = next(train_generator())
print(y_train[0])
print(len(y_train))

0


ValueError: too many values to unpack (expected 2)

In [ ]:
model = Sequential()
model.add(Lambda(lambda x: x/255.0 - 0.5,input_shape=(160,320,3)))
model.add(Cropping2D(cropping=((50,20), (0,0))))
model.add(Convolution2D(24,(5,5),subsample=(2,2),activation='elu'))
model.add(Dropout(.5))
model.add(Convolution2D(36,(5,5),subsample=(2,2),activation='elu'))
model.add(Dropout(.5))
model.add(Convolution2D(48,(5,5),subsample=(2,2),activation='elu'))
model.add(Dropout(.5))
model.add(Convolution2D(64,(3,3),activation='elu'))
model.add(Dropout(.5))
model.add(Convolution2D(64,(3,3),activation='elu'))
model.add(Dropout(.5))
model.add(Flatten())
model.add(Dense(100))
model.add(Dense(50))
model.add(Dense(10))
model.add(Dense(1))

In [ ]:
class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, list_IDs, labels, batch_size=18, dim=(32,32,32), n_channels=1,
                 n_classes=10, shuffle=True):
        'Initialization'
        self.dim = dim
        self.batch_size = batch_size
        self.labels = labels
        self.list_IDs = list_IDs
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]

        # Generate data
        X, y = self.__data_generation(list_IDs_temp)

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X = np.empty((self.batch_size, *self.dim, self.n_channels))
        y = np.empty((self.batch_size), dtype=int)

        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            # Store sample
            X[i,] = np.load('data/' + ID + '.npy')

            # Store class
            y[i] = self.labels[ID]

        return X, keras.utils.to_categorical(y, num_classes=self.n_classes)

In [ ]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Convolution2D, ELU, Flatten, Dropout, Dense, Lambda, MaxPooling2D
from keras.preprocessing.image import img_to_array, load_img
import cv2

rows, cols, ch = 64, 64, 3

TARGET_SIZE = (64, 64)


def augment_brightness_camera_images(image):
    '''
    :param image: Input image
    :return: output image with reduced brightness
    '''

    # convert to HSV so that its easy to adjust brightness
    image1 = cv2.cvtColor(image,cv2.COLOR_RGB2HSV)

    # randomly generate the brightness reduction factor
    # Add a constant so that it prevents the image from being completely dark
    random_bright = .25+np.random.uniform()

    # Apply the brightness reduction to the V channel
    image1[:,:,2] = image1[:,:,2]*random_bright

    # convert to RBG again
    image1 = cv2.cvtColor(image1,cv2.COLOR_HSV2RGB)
    return image1


def resize_to_target_size(image):
    return cv2.resize(image, TARGET_SIZE)


def crop_and_resize(image):
    '''
    :param image: The input image of dimensions 160x320x3
    :return: Output image of size 64x64x3
    '''
    cropped_image = image[55:135, :, :]
    processed_image = resize_to_target_size(cropped_image)
    return processed_image


def preprocess_image(image):
    image = crop_and_resize(image)
    image = image.astype(np.float32)

    #Normalize image
    image = image/255.0 - 0.5
    return image


def get_augmented_row(row):
    steering = row['steering']

    # randomly choose the camera to take the image from
    camera = np.random.choice(['center', 'left', 'right'])

    # adjust the steering angle for left anf right cameras
    if camera == 'left':
        steering += 0.25
    elif camera == 'right':
        steering -= 0.25

    image = load_img("udacity_data/" + row[camera].strip())
    image = img_to_array(image)

    # decide whether to horizontally flip the image:
    # This is done to reduce the bias for turning left that is present in the training data
    flip_prob = np.random.random()
    if flip_prob > 0.5:
        # flip the image and reverse the steering angle
        steering = -1*steering
        image = cv2.flip(image, 1)

    # Apply brightness augmentation
    image = augment_brightness_camera_images(image)

    # Crop, resize and normalize the image
    image = preprocess_image(image)
    return image, steering


def get_data_generator(data_frame, batch_size=32):
    N = data_frame.shape[0]
    batches_per_epoch = N // batch_size

    i = 0
    while(True):
        start = i*batch_size
        end = start+batch_size - 1

        X_batch = np.zeros((batch_size, 64, 64, 3), dtype=np.float32)
        y_batch = np.zeros((batch_size,), dtype=np.float32)

        j = 0

        # slice a `batch_size` sized chunk from the dataframe
        # and generate augmented data for each row in the chunk on the fly
        for index, row in data_frame.loc[start:end].iterrows():
            X_batch[j], y_batch[j] = get_augmented_row(row)
            j += 1

        i += 1
        if i == batches_per_epoch - 1:
            # reset the index so that we can cycle over the data_frame again
            i = 0
        yield X_batch, y_batch


def get_model():
    model = Sequential()
    # model.add(Lambda(preprocess_batch, input_shape=(160, 320, 3), output_shape=(64, 64, 3)))

    # layer 1 output shape is 32x32x32
    model.add(Convolution2D(32, 5, 5, input_shape=(64, 64, 3), subsample=(2, 2), border_mode="same"))
    model.add(ELU())

    # layer 2 output shape is 15x15x16
    model.add(Convolution2D(16, 3, 3, subsample=(1, 1), border_mode="valid"))
    model.add(ELU())
    model.add(Dropout(.4))
    model.add(MaxPooling2D((2, 2), border_mode='valid'))

    # layer 3 output shape is 12x12x16
    model.add(Convolution2D(16, 3, 3, subsample=(1, 1), border_mode="valid"))
    model.add(ELU())
    model.add(Dropout(.4))

    # Flatten the output
    model.add(Flatten())

    # layer 4
    model.add(Dense(1024))
    model.add(Dropout(.3))
    model.add(ELU())

    # layer 5
    model.add(Dense(512))
    model.add(ELU())

    # Finally a single output, since this is a regression problem
    model.add(Dense(1))

    model.compile(optimizer="adam", loss="mse")

    return model

if __name__ == "__main__":
    BATCH_SIZE = 32

    data_frame = pd.read_csv('udacity_data/driving_log.csv', usecols=[0, 1, 2, 3])

    # shuffle the data
    data_frame = data_frame.sample(frac=1).reset_index(drop=True)

    # 80-20 training validation split
    training_split = 0.8

    num_rows_training = int(data_frame.shape[0]*training_split)

    training_data = data_frame.loc[0:num_rows_training-1]
    validation_data = data_frame.loc[num_rows_training:]

    # release the main data_frame from memory
    data_frame = None

    training_generator = get_data_generator(training_data, batch_size=BATCH_SIZE)
    validation_data_generator = get_data_generator(validation_data, batch_size=BATCH_SIZE)

    model = get_model()

    samples_per_epoch = (20000//BATCH_SIZE)*BATCH_SIZE

    model.fit_generator(training_generator, validation_data=validation_data_generator,
                        samples_per_epoch=samples_per_epoch, nb_epoch=3, nb_val_samples=3000)

    print("Saving model weights and configuration file.")

    model.save_weights('model.h5')  # always save your weights after training or during training
    with open('model.json', 'w') as outfile:
        outfile.write(model.to_json())